In [ ]:
from src.genre_classification.F_Basic_Models import Basic_Models
from src.genre_classification.F_Dataset_Downloader import Dataset_Downloader
from src.genre_classification.F_Pretrained_models import Pretrained 
import pandas as pd

# Preparación de los datos

In [ ]:
dataset_downloader = Dataset_Downloader()
train_path, test_path = dataset_downloader(overwrite=False) # Simplemente nos devuelve el path del dataset

train_data = pd.read_csv(train_path.resolve())
test_data = pd.read_csv(test_path.resolve())

train_data.head()

In [ ]:
x_train, y_train = train_data.drop(columns=["genre"]), train_data["genre"]

x_test, y_test = test_data.drop(columns=["genre"]), test_data["genre"]

unique_labels = sorted(list(set(train_data["genre"])))


# Entrenamiento de los Modelos Basicos
Iteramos por cada modelo y entrenamos

In [ ]:
basic_models_name = ['Naive_Bayes', 'LogReg', 'Linear_SVM', 'Random_Forest']

for model_name in basic_models_name:
    model = Basic_Models(model_type= model_name)

    model.fit(x_train, y_train)
    y_hat = model.predict(x_test)
    print(model.evaluate(y_true= y_test, y_hat=y_hat, labels = set(y_test), evaluate_type= "all_metrics"))
    model.save_model(name= model_name)


# Pruebas de Modelos Transformer
Como estos son computacionalmente costosos, no iteramos sobre todos los modelos y simplemente lo seleccionamos, en este caso esta roberta-base, que es el que mejor resultados dio.

In [ ]:
train_texts = train_data["text"].tolist() 
test_texts = test_data["text"].tolist()   

model_transformer = Pretrained(model_type="roberta-base", labels =sorted(list(set(train_data["genre"])))) 

model_transformer.fit(train_texts, train_labels = list(train_data["genre"]), batch_size=16,epochs=3, learning_rate=2e-5, weight_decay=0.01)

predictions = model_transformer.transform(test_texts)

model_transformer.save_model_and_tokenizer()


In [ ]:

from src.genre_classification.F_Compute_Metrics import Compute_Metrics

pred_labels = [model_transformer.id2label[idx] for idx in predictions]

metrics = Compute_Metrics(pred_labels, y_test, unique_labels)
resultados = metrics.compute_all()

print(resultados)